----------------
```{admonition} Learning Objectives
- Work with raster properties and metadata using rioxarray
- Apply transformations and scaling to raster data
- Reproject raster data to different coordinate reference systems (CRS)
- Understand resampling methods and resolution changes
```
----------------



# Working with Raster Properties and Transformations

## Understanding raster properties with rioxarray

Rioxarray provides direct access to important raster properties and metadata through its `rio` accessor. Unlike context managers that set default behaviors, rioxarray uses method chaining and direct property access to control raster operations.

## Purpose of rioxarray's approach

The examples shown in [reading remotely sensed data](f_rs_io.md) demonstrated basic file opening. Rioxarray allows easy control over raster properties like CRS, nodata values, resolution, and spatial bounds through direct method calls and property access.

For instance you might want to check the spatial extent (bounds) of your analysis area. By accessing bounds with rioxarray properties, you can understand your data's spatial coverage and make informed processing decisions.

## How to access raster properties

To access raster properties with rioxarray, use the `rio` accessor on any opened DataArray. For example:

``` python
import rioxarray as rxr

src = rxr.open_rasterio('image.tif')
print(src.rio.crs)        # coordinate reference system
print(src.rio.nodata)     # nodata value
print(src.rio.bounds())   # spatial bounds
print(src.rio.resolution()) # pixel resolution
```

Rioxarray stores raster properties as attributes and methods accessible through the `rio` accessor.

In [ ]:
import rioxarray as rxr

image_path = "../../pygis/data/LC08_L1TP_224078_20200518_20200518_01_RT.TIF"

# Open with rioxarray (no special configuration needed)
src = rxr.open_rasterio(image_path)

# Display basic properties
print('Keyword:', 'crs'.ljust(15), 'Value:', src.rio.crs)
print('Keyword:', 'nodata'.ljust(15), 'Value:', src.rio.nodata)
print('Keyword:', 'shape'.ljust(15), 'Value:', src.shape)
print('Keyword:', 'dtype'.ljust(15), 'Value:', src.dtype)
print('Keyword:', 'resolution'.ljust(15), 'Value:', src.rio.resolution())

### Setting Image Properties
Raster properties can be modified using rioxarray methods. This includes setting nodata values for masking operations and applying pixel value scaling via multiplication.

In [ ]:
import rioxarray as rxr
import numpy as np

image_path = "../../pygis/data/LC08_L1TP_224078_20200518_20200518_01_RT.TIF"

# Set 0 as missing value
src = rxr.open_rasterio(image_path).rio.write_nodata(0)
print("Nodata value:", src.rio.nodata)

# Replace 0 with nan (equivalent to mask_nodata)
src_masked = src.where(src != 0, np.nan)

In [ ]:
import rioxarray as rxr

image_path = "../../pygis/data/LC08_L1TP_224078_20200518_20200518_01_RT.TIF"

# Apply scale factor of 0.0001
src = rxr.open_rasterio(image_path)
src_scaled = src * 0.0001
print("Scale factor applied: 0.0001")

### Coordinate Reference System Transformations

To transform the CRS of raster data, use rioxarray's `rio.reproject()` method. This handles on-the-fly coordinate system transformations using rasterio's virtual warping capabilities.

In [ ]:
import rioxarray as rxr

image_path = "../../pygis/data/LC08_L1TP_224078_20200518_20200518_01_RT.TIF"
proj4 = "+proj=aea +lat_1=-5 +lat_2=-42 +lat_0=-32 +lon_0=-60 +x_0=0 +y_0=0 +ellps=aust_SA +units=m +no_defs"

# Without transformation
src = rxr.open_rasterio(image_path)
print("Original CRS:", src.rio.crs)

# With CRS transformation
src_transformed = src.rio.reproject(proj4)
print("Transformed CRS:", src_transformed.rio.crs)

### Resolution and CRS Transformation Combined

Multiple transformation parameters can be combined in a single reproject operation. In this example, the raster CRS is transformed from UTM to Albers Equal Area with a resampled cell size of 100m x 100m.

In [ ]:
import rioxarray as rxr
from rasterio.enums import Resampling

image_path = "../../pygis/data/LC08_L1TP_224078_20200518_20200518_01_RT.TIF"
proj4 = "+proj=aea +lat_1=-5 +lat_2=-42 +lat_0=-32 +lon_0=-60 +x_0=0 +y_0=0 +ellps=aust_SA +units=m +no_defs"

# Without transformation
src = rxr.open_rasterio(image_path)
print("Original resolution:", src.rio.resolution())

# With CRS and resolution transformation
src_transformed = src.rio.reproject(proj4, resolution=(100, 100), resampling=Resampling.bilinear)
print("New resolution:", src_transformed.rio.resolution())

### Spatial Subsetting by Bounds

To subset a raster by spatial bounds, use rioxarray's `rio.clip_box()` method. Bounds should be specified as a tuple of (left, bottom, right, top) coordinates in the raster's coordinate system.

In [ ]:
import rioxarray as rxr

image_path = "../../pygis/data/LC08_L1TP_224078_20200518_20200518_01_RT.TIF"
bounds = (724634.17, -2806501.39, 737655.48, -2796221.42)

# Without clipping
src = rxr.open_rasterio(image_path)
print("Original bounds:", src.rio.bounds())

# With bounds clipping
src_clipped = src.rio.clip_box(*bounds)
print("Clipped bounds:", src_clipped.rio.bounds())

### Raster Alignment and Reference Matching

Rioxarray provides `rio.reproject_match()` to align one raster to match another raster's spatial properties (bounds, CRS, and resolution). This ensures consistent spatial alignment between datasets.

In [ ]:
import rioxarray as rxr

image_path = "../../pygis/data/LC08_L1TP_224078_20200518_20200518_01_RT.TIF"
ref_image_path = "../../pygis/data/LC08_L1TP_224078_20200518_20200518_01_RT.TIF"  # Use same file as reference

# Without reference matching
src = rxr.open_rasterio(image_path)
print("Original bounds:", src.rio.bounds())

# Load reference image
ref_img = rxr.open_rasterio(ref_image_path)
print("Reference bounds:", ref_img.rio.bounds())

# Match to reference (in this case, same image)
src_matched = src.rio.reproject_match(ref_img)
print("Matched bounds:", src_matched.rio.bounds())

(f_rs_crs_sensors)=
### Band Naming and Sensor Configurations

Since rasters are opened as DataArrays, bands have coordinate names. By default, bands are numbered starting from 1. However, it's often more intuitive to assign meaningful names corresponding to wavelengths or sensor bands using xarray's `assign_coords()` method.

In [ ]:
import rioxarray as rxr

image_path = "../../pygis/data/LC08_L1TP_224078_20200518_20200518_01_RT.TIF"

# Without band names
src = rxr.open_rasterio(image_path)
print("Original bands:", src.band.values)
print("Number of bands:", len(src.band))

# With BGR sensor configuration (assign band names matching actual number of bands)
if len(src.band) == 3:
    src_bgr = src.assign_coords(band=['blue', 'green', 'red'])
    print("BGR bands:", src_bgr.band.values)
else:
    # For files with more bands, adjust accordingly
    band_names = ['blue', 'green', 'red', 'nir', 'swir1', 'swir2', 'thermal'][:len(src.band)]
    src_bgr = src.assign_coords(band=band_names)
    print("BGR bands:", src_bgr.band.values)

### Common Sensor Configurations

Different sensors have standard band configurations. While rioxarray doesn't have built-in sensor definitions, you can create your own band naming conventions based on common sensor specifications.

In [ ]:
# Since rioxarray doesn't have built-in sensor definitions, 
# we'll create a simple sensor dictionary
sensor_names = {
    'l8': 'Landsat 8',
    's2': 'Sentinel-2',
    'bgr': 'Blue-Green-Red configuration',
    'rgb': 'Red-Green-Blue configuration'
}

for sensor_name, description in sensor_names.items():
    print('{}: {}'.format(sensor_name.ljust(15), description))

The following table shows common sensor band configurations that can be implemented using rioxarray's `assign_coords()` method:

```{list-table}
:header-rows: 1
:widths: 25 75
:header-rows: 1

* - Abbreviated Name
  - Description

* - 'rgb'
  - red, green, and blue

* - 'rgbn'
  - red, green, blue, and NIR

* - 'bgr'
  - blue, green, and red

* - 'bgrn'
  - blue, green, red, and NIR

* - 'l5'
  - Landsat 5 Thematic Mapper (TM)

* - 'l7'
  - Landsat 7 Enhanced Thematic Mapper Plus (ETM+) without panchromatic and thermal bands

* - 'l7th'
  - Landsat 7 Enhanced Thematic Mapper Plus (ETM+) with thermal band

* - 'l7mspan'
  - Landsat 7 Enhanced Thematic Mapper Plus (ETM+) with panchromatic band

* - 'l7pan'
  - Landsat 7 panchromatic band

* - 'l8'
  - Landsat 8 Operational Land Imager (OLI) and Thermal Infrared Sensor (TIRS) without panchromatic and thermal bands

* - 'l8l7'
  - Landsat 8 Operational Land Imager (OLI) and Thermal Infrared Sensor (TIRS) with 6 Landsat 7-like bands

* - 'l8l7mspan'
  - Landsat 8 Operational Land Imager (OLI) and panchromatic band with 6 Landsat 7-like bands

* - 'l8th'
  - Landsat 8 Operational Land Imager (OLI) and Thermal Infrared Sensor (TIRS) with thermal band

* - 'l8pan'
  - Landsat 8 panchromatic band

* - 's2'
  - Sentinel 2 Multi-Spectral Instrument (MSI) without 3 60m bands (coastal, water vapor, cirrus)

* - 's2f'
  - Sentinel 2 Multi-Spectral Instrument (MSI) with 3 60m bands (coastal, water vapor, cirrus)

* - 's2l7'
  - Sentinel 2 Multi-Spectral Instrument (MSI) with 6 Landsat 7-like bands

* - 's210'
  - Sentinel 2 Multi-Spectral Instrument (MSI) with 4 10m (visible + NIR) bands

* - 's220'
  - Sentinel 2 Multi-Spectral Instrument (MSI) with 6 20m bands

* - 's2cloudless'
  - Sentinel 2 Multi-Spectral Instrument (MSI) with 10 bands for s2cloudless

* - 'ps'
  - PlanetScope with 4 (visible + NIR) bands

* - 'qb'
  - Quickbird with 4 (visible + NIR) bands

* - 'ik'
  - IKONOS with 4 (visible + NIR) bands
```